<a href="https://colab.research.google.com/github/alexganea7/Mini-projects-for-university-AI-class/blob/main/Ganea_Alexandru_Gabriel%3B_Grupa_311%3B_Mini_proiect_jocuri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dame vs AI**




---


In continuare voi prezenta implementarea jocului "Dame" folosind algoritmul "Minimax" pentru a calcula mutarile agentului inteligent.




---



In [ ]:
# Dependencies:
!pip install colorama # pentru output colorat
import time # compar algoritmii
from colorama import Fore, Style, Back # output colorat
import copy
import numpy as np

# **1. Regulile jocului**

Tabla este reprezentata din 8x8 casute de culori alb si negru. Piesele sunt reprezentate din cate 12 discuri de culori diferite. Fiecare piesa se aseaza in ordine doar pe patratele negre. Mutarile se fac doar pe diagonala in fata doar o patratica(exceptie facand regele, care poate de asemenea sa se mute inapoi). O pisa normala poate deveni rege daca ajunge la margine pe partea adversarului de tabla. O piesa este eliminata daca o piesa adversara "sare" peste aceasta saltul fiind facut tot pe diagonala. Daca dupa un astfel de "salt" aceeasi piesa poate sari din nou(si este obligatoriu) peste alte piese adversare daca este posibil. Daca este posibil un salt, acesta este obligatoriu sa fie facut! Meciul se opreste cand un jucator nu mai poate face nici o miscare sau cand acesta ramane fara piese.

# **2. Reprezentarea tablei de joc**

Pentru tabla de joc vom folosi o matrice in care piesele jucatorului sunt reprezentate cu **'a'**, respectiv **'A'** pentru rege. Piesele agentului inteligent fiind reprezentate cu **'n'**, respectiv **'N'**.

Exemplu de tabla:


```
board = [
        ['','n','','n','','n','','n'],
        ['n','','n','','n','','n',''],
        ['','n','','n','','n','','n'],
        ['','','','','','','',''],
        ['','','','','','','',''],
        ['a','','a','','a','','a',''],
        ['','a','','a','','a','','a'],
        ['a','','a','','a','','a',''],
        ]
```



In [ ]:
board = [
        ['','n','','n','','n','','n'],
        ['n','','n','','n','','n',''],
        ['','n','','n','','n','','n'],
        ['','','','','','','',''],
        ['','','','','','','',''],
        ['a','','a','','a','','a',''],
        ['','a','','a','','a','','a'],
        ['a','','a','','a','','a',''],
        ]


Pentru a face mai usor de urmarit jocul, vom implementa o functie de afisare (colorta pentru ca arata mai frumos asa). Pentru a colora inputul am folosit libraria colorama.

In [ ]:
def printBoard(board):
    print('    0    1    2    3    4    5    6    7')
    print('  ----------------------------------------')
    print("0|", end='')
    index = 1
    for i1, b in enumerate(board):
        for i2, bb in enumerate(b):
            if(bb != ''):
                print(Back.BLACK+Fore.LIGHTRED_EX + Style.BRIGHT+' ',bb, end='  ') if bb=='a' or bb=='A' else print(Back.BLACK + Fore.LIGHTBLUE_EX + Style.BRIGHT + ' ',bb, end='  ')
            else:
                if(((i1%2) == 0 and (i2%2) == 1) or ((i1%2) == 1 and (i2%2) == 0)):
                    print(Style.RESET_ALL + Back.BLACK + '  -  ', end='')
                else:
                    print(Style.RESET_ALL + Back.WHITE + '  -  ', end='')
        print(Style.RESET_ALL + f'|\n{index}|'  if index <8 else Style.RESET_ALL + '|\n ', end='')
        index+=1
    print(' ----------------------------------------\n')
printBoard(board)

    0    1    2    3    4    5    6    7
  ----------------------------------------
0|  -    n    -    n    -    n    -    n  |
1|  n    -    n    -    n    -    n    -  |
2|  -    n    -    n    -    n    -    n  |
3|  -    -    -    -    -    -    -    -  |
4|  -    -    -    -    -    -    -    -  |
5|  a    -    a    -    a    -    a    -  |
6|  -    a    -    a    -    a    -    a  |
7|  a    -    a    -    a    -    a    -  |
  ----------------------------------------



# **3. Functia succesor**

Succesorii, adica urmatoarele pozitii posibile, pot fi de 2 feluri. In cazul in care o piesa poate fi luata de adversar, vom numi acest succesor ca fiind un salt. In cazul in care nu exista nici un salt, vom numii succesorii ca mutari normale. Mutarile normale reprezinta mutarea unei piese pe diagonala in fata(in spate din perspectiva noastra pentru agentul inteligent) stanga sau dreapta. In cazul regelui putem sa ne mutam pe diagonala in orice directie. In cazul salturilor trebuie sa verificam daca diagonala este "blocata" de o piesa adversara si daca diagonala imediat urmatoare este libera. Vom folosi functii separate pentru succesorii agentului inteligent si succesorii nostrii. Rezultatul functiei va fi un tuplu de forma (1,2,3,4, 'a') care inseamna: "piesa (1,2) se duce pe pozitia (3,4) si are tipul 'a'", iar in cazul salturilor, vom specifica si peste ce piesa are pentru a o putea elimina ulterior.

In [ ]:
def verificaSarituraAI(board, i, j, isarit, jsarit, inou, jnou):
    if inou > 7 or jnou < 0 or jnou > 7 or inou < 0:
        return False
    if board[isarit][jsarit] != 'a' and board[isarit][jsarit] != 'A':
        return False
    if board[inou][jnou] != '':
        return False
    return True

def verificaSarituraOM(board, i, j, isarit, jsarit, inou, jnou):
    if inou > 7 or jnou < 0 or jnou > 7 or inou < 0:
        return False
    if board[isarit][jsarit] != 'n' and board[isarit][jsarit] != 'N':
        return False
    if board[inou][jnou] != '':
        return False
    return True

def verificaMutare(board, i, j, inou, jnou):
    if inou > 7 or jnou < 0 or jnou > 7 or inou<0:
        return False
    if board[inou][jnou] != '':
        return False
    return True

def sarituriAI(board): #adica N
    sarituri = []
    for i in range(8):
        for j in range(8):
            if(board[i][j]=='n'):
                if(verificaSarituraAI(board, i, j, i+1, j+1, i+2, j+2)):
                    sarituri.append((i,j,i+2,j+2, i+1, j+1, 'n'))
                if(verificaSarituraAI(board, i, j, i+1, j-1, i+2, j-2)):
                    sarituri.append((i,j,i+2,j-2, i+1, j-1, 'n'))
            if(board[i][j]=='N'):
                if(verificaSarituraAI(board, i, j, i+1, j+1, i+2, j+2)):
                    sarituri.append((i,j,i+2,j+2, i+1, j+1, 'N'))
                if(verificaSarituraAI(board, i, j, i+1, j-1, i+2, j-2)):
                    sarituri.append((i,j,i+2,j-2, i+1, j-1, 'N'))
                if(verificaSarituraAI(board, i, j, i-1, j+1, i-2, j+2)):
                    sarituri.append((i,j,i-2,j+2, i-1, j+1, 'N'))
                if(verificaSarituraAI(board, i, j, i-1, j-1, i-2, j-2)):
                    sarituri.append((i,j,i-2,j-2, i-1, j-1, 'N'))
    return sarituri

def sarituriOM(board): #adica A
    sarituri = []
    for i in range(8):
        for j in range(8):
            if(board[i][j]=='a'):
                if(verificaSarituraOM(board, i, j, i-1, j+1, i-2, j+2)):
                    sarituri.append((i,j,i-2,j+2, i-1, j+1, 'a'))
                if(verificaSarituraOM(board, i, j, i-1, j-1, i-2, j-2)):
                    sarituri.append((i,j,i-2,j-2, i-1, j-1, 'a'))
            if(board[i][j]=='A'):
                if(verificaSarituraOM(board, i, j, i-1, j+1, i-2, j+2)):
                    sarituri.append((i,j,i-2,j+2, i-1, j+1, 'A'))
                if(verificaSarituraOM(board, i, j, i-1, j-1, i-2, j-2)):
                    sarituri.append((i,j,i-2,j-2, i-1, j-1, 'A'))
                if(verificaSarituraOM(board, i, j, i+1, j+1, i+2, j+2)):
                    sarituri.append((i,j,i+2,j+2, i+1, j+1, 'A'))
                if(verificaSarituraOM(board, i, j, i+1, j-1, i+2, j-2)):
                    sarituri.append((i,j,i+2,j-2, i+1, j-1, 'A'))

    return sarituri

def succesoriAI(board):
    succesori = []
    for i in range(8):
        for j in range(8):
            if(board[i][j]=='n'):
                if(verificaMutare(board,i,j,i+1, j+1)):
                    succesori.append((i,j, i+1, j+1, 'n'))
                if(verificaMutare(board,i,j,i+1, j-1)):
                    succesori.append((i,j,i+1, j-1, 'n'))
            if(board[i][j]=='N'):
                if(verificaMutare(board,i,j,i+1,j-1)):
                    succesori.append((i,j, i+1, j-1, 'N'))
                if(verificaMutare(board,i,j,i+1,j+1)):
                    succesori.append((i,j, i+1, j+1, 'N'))
                if(verificaMutare(board,i,j,i-1,j-1)):
                    succesori.append((i,j, i-1, j-1, 'N'))
                if(verificaMutare(board,i,j,i-1,j+1)):
                    succesori.append((i,j, i-1, j+1, 'N'))
    return succesori

def succesoriOM(board):
    succesori = []
    for i in range(8):
        for j in range(8):
            if(board[i][j]=='a'):
                if(verificaMutare(board,i,j,i-1, j+1)):
                    succesori.append((i,j, i-1, j+1, 'a'))
                if(verificaMutare(board,i,j,i-1, j-1)):
                    succesori.append((i,j,i-1, j-1, 'a'))
            if(board[i][j]=='A'):
                if(verificaMutare(board,i,j,i+1,j-1)):
                    succesori.append((i,j, i+1, j-1, 'A'))
                if(verificaMutare(board,i,j,i+1,j+1)):
                    succesori.append((i,j, i+1, j+1, 'A'))
                if(verificaMutare(board,i,j,i-1,j-1)):
                    succesori.append((i,j, i-1, j-1, 'A'))
                if(verificaMutare(board,i,j,i-1,j+1)):
                    succesori.append((i,j, i-1, j+1, 'A'))
    return succesori

**Observatii:**


*   Pentru a fi mai usor de citit codul(sa nu am prea multe if-uri) am folosit functii ajutatoare pentru a testa daca o mutare este posibila
*   Am impartit functia succesori in doua deoarece daca un salt este posibil nu avem voie sa facem o mutare normala(saltul fiind obligatoriu). Aceasta decizie va face implementarea mai usoara si mai usor de citit. (vom verifica mutarile doar daca functia de salturi disponibile nu returneaza nimic)



In [ ]:
print("Sarituri player:", sarituriOM(board))
print("Sarituri AI:", sarituriAI(board))
print("Mutari player:", succesoriOM(board))
print("Mutari AI:", succesoriAI(board))


board2 = [
        ['','','','','','','',''],
        ['','','','','','','',''],
        ['','','','','','','',''],
        ['','','','','','','',''],
        ['','','','n','','n','',''],
        ['','','a','','a','','',''],
        ['','','','','','','',''],
        ['','','','','','','',''],
        ]
print("TEST PE ALTA TABLA:\n")
print("Sarituri player:", sarituriOM(board2))
print("Sarituri AI:", sarituriAI(board2))

Sarituri player: []
Sarituri AI: []
Mutari player: [(5, 0, 4, 1, 'a'), (5, 2, 4, 3, 'a'), (5, 2, 4, 1, 'a'), (5, 4, 4, 5, 'a'), (5, 4, 4, 3, 'a'), (5, 6, 4, 7, 'a'), (5, 6, 4, 5, 'a')]
Mutari AI: [(2, 1, 3, 2, 'n'), (2, 1, 3, 0, 'n'), (2, 3, 3, 4, 'n'), (2, 3, 3, 2, 'n'), (2, 5, 3, 6, 'n'), (2, 5, 3, 4, 'n'), (2, 7, 3, 6, 'n')]
TEST PE ALTA TABLA:

Sarituri player: [(5, 2, 3, 4, 4, 3, 'a'), (5, 4, 3, 6, 4, 5, 'a'), (5, 4, 3, 2, 4, 3, 'a')]
Sarituri AI: [(4, 3, 6, 5, 5, 4, 'n'), (4, 3, 6, 1, 5, 2, 'n'), (4, 5, 6, 3, 5, 4, 'n')]


**Observatie 2:**
Functiile noastre returneaza niste tupluri deci va fi problematic in algoritm sa testeze succesorii acestor tupluri. Pentru a rezolva aceasta problema trebuie sa facem o functie care transforma tuplul intr-o tabla noua. (Doua functii, una pentru salt, una pentru mutare normala). In aceste functii vom face si "avansarea" unei piese la gradul de rege.

In [ ]:
def creeazaBoardSalt(board, succesor):
    copie = copy.deepcopy(board)
    iv, jv, inou, jnou, isters, jsters, tip = succesor
    copie[iv][jv]=''
    if tip == 'n' and inou == 7:
        tip = 'N'
    if tip == 'a' and inou == 0:
        tip = 'A'
    copie[inou][jnou]=tip
    copie[isters][jsters]=''
    return copie

def creeazaBoardNormal(board, succesor):
    copie = copy.deepcopy(board)
    iv, jv, inou, jnou, tip = succesor
    copie[iv][jv]=''
    if tip == 'n' and inou == 7:
        tip = 'N'
    if tip == 'a' and inou == 0:
        tip = 'A'
    copie[inou][jnou]=tip
    return copie

mutare = succesoriOM(board)[0] # am luat o mutare random trebuie rulat si codul anterior
saritura = sarituriAI(board2)[0] # am luat o saritura random trebuie rulat si codul anterior
print(mutare, saritura, '\n')
boardtest1 = creeazaBoardNormal(board, mutare)
boardtest2 = creeazaBoardSalt(board2, saritura)
printBoard(boardtest1)
printBoard(boardtest2)

(5, 0, 4, 1, 'a') (4, 3, 6, 5, 5, 4, 'n') 

    0    1    2    3    4    5    6    7
  ----------------------------------------
0|  -    n    -    n    -    n    -    n  |
1|  n    -    n    -    n    -    n    -  |
2|  -    n    -    n    -    n    -    n  |
3|  -    -    -    -    -    -    -    -  |
4|  -    a    -    -    -    -    -    -  |
5|  -    -    a    -    a    -    a    -  |
6|  -    a    -    a    -    a    -    a  |
7|  a    -    a    -    a    -    a    -  |
  ----------------------------------------

    0    1    2    3    4    5    6    7
  ----------------------------------------
0|  -    -    -    -    -    -    -    -  |
1|  -    -    -    -    -    -    -    -  |
2|  -    -    -    -    -    -    -    -  |
3|  -    -    -    -    -    -    -    -  |
4|  -    -    -    -    -    n    -    -  |
5|  -    -    a    -    -    -    -    -  |
6|  -    -    -    -    -    n    -    -  |
7|  -    -    -    -    -    -    -    -  |
  --------------------------------------

# **4. Finalul jocului**

Cum am precizat in regulile jocului, un meci se termina cand ori un jucator ramane fara mutari, ori acesta nu mai are piese disponibile. Pentru a implementa o functie care verifica daca s-a terminat jocul, trebuie doar sa iteram prin matrice, sa numaram piesele si sa le comparam cu 0. Pentru a verifica daca exista mutari posibile, apelam functiile succesor si le verificam lungimea.

In [ ]:
def final(board):
    sumN = 0
    sumA = 0
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 'n' or board[i][j] == 'N':
                sumN+=1
            if board[i][j] == 'a' or board[i][j] == 'A':
                sumA+=1
    if(len(succesoriAI(board))==0 and len(sarituriAI(board)) == 0):
        return 'a'
    if(len(succesoriOM(board))==0 and len(sarituriOM(board)) == 0):
        return 'n'
    if(sumN==0):
        return 'a'
    if(sumA==0):
        return 'n'
    return 'continua'

print("Stare tabla 1:", final(board))
board3 = [
         ['','','','','','','',''],
         ['','','','','','','',''],
         ['','','','','','n','',''],
         ['','','','','','','',''],
         ['','','','n','','','',''],
         ['','','','','','','',''],
         ['','','','','','','',''],
         ['','','','','','','N',''],
         ]
print("Stare tabla 3:", final(board3))

# 3 outputuri posibile: n sau a inseamna ca AI-ul respectiv player-ul au castigat, iar continua inseamna ca nu s-a terminat inca jocul

Stare tabla 1: continua
Stare tabla 3: n


# **5. Calculul agentului inteligent**

Pentru a calcula mutarile AI-ului, vom folosi algoritmul "minimax" respectiv varianta sa optimizata, "alpha-beta pruning". Pentru ca exista un numar extrem de mare de posibilitati pana la finalul unui joc, nu putem calcula pana la final toate mutarile posibile deci trebuie sa avem o conditie de oprire urmata de o functie de evaluare a starii pentru a putea face alegerea optima. Pentru aceasta avem nevoie sa limitam in adancime algoritmul, comparand numarul de succesori pe nivel cu o adancime maxima data ca variabila (cu cat e mai mare, cu atat agentul inteligent face alegeri mai bune dar necesita mai multe resurse).

## **5.1. Functia euristica**
Pentru a evalua starea unei table, vom folosi o euristica cu 3 parametrii: numarul de piese normale, numarul de regi si numarul de piese in partea inferioara jucatorului a tablei. Pentru fiecare vom avea ponderi diferite. Pentru aceasta functie am folosit ponderile de 5 pentru o piesa normala, 8 pentru un rege si 4 pentru o piesa aflata in partea inferioara a tablei. (Valorile respective au fost gasite in urmatorul articol [link](https://github.com/kevingregor/Checkers/blob/master/Final%20Project%20Report.pdf).)

In [ ]:
def euristica(board):
    nrn, nra, nrN, nrA = 0,0,0,0 # numar de piese a,n= normale, A,N = regi
    nrspate, nrfata = 0, 0
    # voi folosi (inspirandu-ma de aici https://github.com/kevingregor/Checkers/blob/master/Final%20Project%20Report.pdf)) o euristica cu 3 parametrii

    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 'n':
                nrn+=1
            if board[i][j] == 'a':
                nra+=1
            if board[i][j] == 'N':
                nrN+=2
            if board[i][j] == 'A':
                nrA+=2
    #verific cate piese sunt in jumatea player-ului/ai-ului
    for i in range(len(board)//2, 8):
        for j in range(len(board)//2, 8):
            if(board[i][j] in ('a','A')):
                nrspate+=1
    for i in range(0,len(board)//2):
        for j in range(0,len(board)//2):
            if(board[i][j] in ('n','N')):
                nrfata+=1

    p = [5, 8, 4] #ponderi
    return p[0]*(nra-nrn) + p[1]*(nrA-nrN) + p[2]*(nrspate-nrfata)


board4 = [
         ['','A','','n','','n','','n'],
         ['','','','','','','',''],
         ['','','','','','','','n'],
         ['','','','','','','',''],
         ['','','','','','','',''],
         ['','','','','','','',''],
         ['','a','','a','','a','',''],
         ['a','','a','','a','','a',''],
         ]
printBoard(board4)
print("Valoare tabla:", euristica(board4))

    0    1    2    3    4    5    6    7
  ----------------------------------------
0|  -    A    -    n    -    n    -    n  |
1|  -    -    -    -    -    -    -    -  |
2|  -    -    -    -    -    -    -    n  |
3|  -    -    -    -    -    -    -    -  |
4|  -    -    -    -    -    -    -    -  |
5|  -    -    -    -    -    -    -    -  |
6|  -    a    -    a    -    a    -    -  |
7|  a    -    a    -    a    -    a    -  |
  ----------------------------------------

Valoare tabla: 39


# **5.2. Minimax**

Algoritmul este clasic cu o mica modificare. Deoarece saltul este obligatoriu facut, intai vad daca exista vreun salt posibil, in caz contrar verific toate mutarile normale. Avem doua conditii de oprire, ori unul dintre jucatori castiga, ori se atinge adancimea maxima. In cazul in care unul dintre jucatori castiga vom returna un numar mare, respectiv mic pentru a depasi orice valoare posibila din functia euristica. In cazul in care s-a atins adancimea maxima, returnam evaluarea starii curente folosind functia euristica prezentata anterior.

In [ ]:
def minimaxLimitat(board, player, adancime, adancimeMax):
  rezultat = final(board)
  if rezultat != 'continua':
      if rezultat == 'a':
          return 1000 - adancime
      else:
          return -1000 + adancime
  if(adancime>=adancimeMax):
      return euristica(board)
  global nr_apeluri

  if player == 'a':
    maxim = -np.inf
    sarituri = sarituriOM(board)
    if(len(sarituri)):
        for fiu in sarituri:
            fiu = creeazaBoardSalt(board, fiu)
            nr_apeluri += 1
            val = minimaxLimitat(fiu, 'n', adancime+1, adancimeMax)
            maxim = max(maxim, val)
        return maxim
    else:
        for fiu in succesoriOM(board):
            fiu = creeazaBoardNormal(board, fiu)
            nr_apeluri += 1
            val = minimaxLimitat(fiu, 'n', adancime+1, adancimeMax)
            maxim = max(maxim, val)
        return maxim
  else:
    minim = np.inf
    sarituri = sarituriAI(board)
    if(len(sarituri)):
        for fiu in sarituri:
            fiu = creeazaBoardSalt(board, fiu)
            nr_apeluri += 1
            val = minimaxLimitat(fiu, 'a', adancime+1, adancimeMax)
            minim = min(minim, val)
        return minim
    else:
        for fiu in succesoriAI(board):
            fiu = creeazaBoardNormal(board, fiu)
            nr_apeluri += 1
            val = minimaxLimitat(fiu, 'a', adancime+1, adancimeMax)
            minim = min(minim, val)
        return minim
nr_apeluri = 0 #vorbesc de el mai tarziu
print(minimaxLimitat(board, 'n', 0, 3)) #return -1 asta ar insemna ca AI-ul poate fi in avantaj in urmatoarele 3(adancime max) mutari

-1


# **5.3. Alpha-beta pruning**

Pentru o adancime de la 4 in sus, minimax devine din ce in ce mai incet. O optimizare al acestui algoritm il reprezinta algoritmul "alpha-beta pruning" care ignora ramurile care nu pot duce la o stare optima. Ca si in cazul anterior, trebuie sa testam intai salturile dupa care mutarile normale in cazul in care nu exista salturi.

In [ ]:
def minimaxAlphaBetaLimitat(board, player, adancime, adancimeMax):
  if(player == 'a'):
    return maxValueLimitat(board, adancime, -np.inf, np.inf, adancimeMax)
  else:
    return minValueLimitat(board, adancime, -np.inf, np.inf, adancimeMax)

def minValueLimitat(board, adancime, alpha, beta, adancimeMax): # recursiv
    global nr_apeluri
    rezultat = final(board)
    if rezultat != 'continua':
        if rezultat == 'a':
            return 1000 - adancime
        else:
            return -1000 + adancime
    if(adancime>=adancimeMax):
        return euristica(board)

    v = np.inf
    sarituri = sarituriAI(board)
    if(len(sarituri)):
        for fiu in sarituri:
            nr_apeluri += 1
            fiu = creeazaBoardSalt(board, fiu)
            v = min(v, maxValueLimitat(fiu, adancime+1, alpha, beta, adancimeMax))
            if v <= alpha:
                return v
            beta = min(beta,v)
        return v
    else:
        for fiu in succesoriAI(board):
            nr_apeluri += 1
            fiu = creeazaBoardNormal(board, fiu)
            v = min(v, maxValueLimitat(fiu, adancime+1, alpha, beta, adancimeMax))
            if v <= alpha:
                return v
            beta = min(beta,v)
        return v

def maxValueLimitat(board, adancime, alpha, beta, adancimeMax): # recursiv
    global nr_apeluri
    rezultat = final(board)
    if rezultat != 'continua':
        if rezultat == 'a':
            return 1000 - adancime
        else:
            return -1000 + adancime
    if(adancime>=adancimeMax):
        return euristica(board)

    v = -np.inf
    sarituri = sarituriOM(board)
    if(len(sarituri)):
        for fiu in sarituri:
            nr_apeluri += 1
            fiu = creeazaBoardSalt(board, fiu)
            v = max(v, minValueLimitat(fiu, adancime+1, alpha, beta, adancimeMax))
            if v >= beta:
                return v
            alpha = max(alpha,v)
        return v
    else:
        for fiu in succesoriOM(board):
            nr_apeluri += 1
            fiu = creeazaBoardNormal(board, fiu)
            v = max(v, minValueLimitat(fiu, adancime+1, alpha, beta, adancimeMax))
            if v >= beta:
                return v
            alpha = max(alpha,v)
        return v

nr_apeluri = 0 #vorbesc de el mai tarziu
print(minimaxAlphaBetaLimitat(board, 'n', 0, 3)) #return -1 asta ar insemna ca AI-ul poate fi in avantaj in urmatoarele 3(adancime max) mutari

-1


# **5.4. Comparatie intre cei 2 algoritmi**
Algoritmul "alpha-beta pruning" de la o adancime anume, devine exponential mai rapid decat "minimax". Diferenta se vede clar si in timpul de executie al functiilor.

In [ ]:
import time
nr_apeluri = 0
start = time.time()
minimaxAlphaBetaLimitat(board, 'n', 0, 8)
end = time.time()
print("Numar apeluri alpha-beta:", nr_apeluri, " timp rulare:", end-start, "secunde") #10402 pt adancime 7

nr_apeluri = 0
start = time.time()
minimaxLimitat(board, 'n', 0, 7 )
end = time.time()
print("Numar apeluri minimax:", nr_apeluri, " timp rulare:", end-start, "secunde") #225974 pt adancime 7

Numar apeluri alpha-beta: 18580  timp rulare: 2.763939142227173 secunde


KeyboardInterrupt: ignored

# **6. Probleme de implementare**

O problema pe care am intalnit-o in implementare a fost problema salturilor multiple. In cazul in care un salt duce catre un alt salt posibil cu aceeasi piesa trebuie facut si acel salt. Pentru a rezolva aceasta problema am considerat noul salt ca o urmatoare runda fara a schimba randul playerului. Am limitat urmatoarele mutari posibile doar la salturile specifice acelei piese.

Pentru mutarile agentului inteligent am luat o valoare maxima foarte mare si am incercat sa gasesc cel mai mic output din functia de minimax comparand mereu valoarea optima momentana cu fiecare dintre succesorii posibili.

Pentru a face mutarile jucatorului uman, trebuie precizate toate detaliile precum pozitia piesei initiale, pozitia unde vrem sa mutam cat si tipul piesei('a' sau 'A'). In cazul saltului trebuie precizat si piesa peste care sarim.

Pentru a "porni" jocul am folosit o functie care se ocupa cu mutarile jucatorului cat si agentului inteligent. Aceasta are ca parametrii cine este playerul, algoritmul folosit si adancimea maxima.

In [ ]:
def printBoard(board):
    print('    0    1    2    3    4    5    6    7')
    print('  ----------------------------------------')
    print("0|", end='')
    index = 1
    for i1, b in enumerate(board):
        for i2, bb in enumerate(b):
            if(bb != ''):
                print(Back.BLACK+Fore.LIGHTRED_EX + Style.BRIGHT+' ',bb, end='  ') if bb=='a' or bb=='A' else print(Back.BLACK + Fore.LIGHTBLUE_EX + Style.BRIGHT + ' ',bb, end='  ')
            else:
                if(((i1%2) == 0 and (i2%2) == 1) or ((i1%2) == 1 and (i2%2) == 0)):
                    print(Style.RESET_ALL + Back.BLACK + '  -  ', end='')
                else:
                    print(Style.RESET_ALL + Back.WHITE + '  -  ', end='')
        print(Style.RESET_ALL + f'|\n{index}|'  if index <8 else Style.RESET_ALL + '|\n ', end='')
        index+=1
    print(' ----------------------------------------\n')
    print('scor tabla:', euristica(board), '\n')
#l-am rescris aici pentru a afisa si scorul tablei
def joc(player, algoritm, adancimeMax):
    board = [
        ['','n','','n','','n','','n'],
        ['n','','n','','n','','n',''],
        ['','n','','n','','n','','n'],
        ['','','','','','','',''],
        ['','','','','','','',''],
        ['a','','a','','a','','a',''],
        ['','a','','a','','a','','a'],
        ['a','','a','','a','','a',''],
        ]

    printBoard(board)
    while(final(board)=='continua'):
        if(player=='a'):
            sarituri = sarituriOM(board)
            if(len(sarituri) == 0):
                mutari = succesoriOM(board)
                print("Mutari disponibile:", mutari)
                exemplu = ' '.join(str(element) for element in mutari[0])
                x = input(f"alege mutare (exemplu: {exemplu}): ")
                print('\n')
                x = x.split()
                tip = x[-1]
                x = [int(nr) for nr in x[:-1]]
                x.append(tip)
                x = tuple(x)
                while(x not in mutari):
                    print("MUTARE INVALIDA")
                    exemplu = ' '.join(str(element) for element in mutari[0])
                    x = input(f"alege mutare (exemplu: {exemplu}): ")
                    print('\n')
                    x = x.split()
                    tip = x[-1]
                    x = [int(nr) for nr in x[:-1]]
                    x.append(tip)
                    x = tuple(x)
                board = creeazaBoardNormal(board, x)
                printBoard(board)
                print("Ai mutat: ", x, '\n')
            else:
                while(len(sarituri)):
                    print("Sarituri obligatorii:", sarituri)
                    exemplu = ' '.join(str(element) for element in sarituri[0])
                    x = input(f"alege mutare (exemplu: {exemplu}): ")
                    print('\n')
                    x = x.split()
                    tip = x[-1]
                    x = [int(nr) for nr in x[:-1]]
                    x.append(tip)
                    x = tuple(x)
                    while(x not in sarituri):
                        print("SALT INVALID")
                        exemplu = ' '.join(str(element) for element in sarituri[0])
                        x = input(f"alege saritura (exemplu: {exemplu}): ")
                        print('\n')
                        x = x.split()
                        tip = x[-1]
                        x = [int(nr) for nr in x[:-1]]
                        x.append(tip)
                        x = tuple(x)
                    board = creeazaBoardSalt(board, x)
                    sarituri = sarituriOM(board)
                    for saritura in sarituri:
                        if (saritura[0], saritura[1]) != (x[2], x[3]):
                            sarituri.remove(saritura)
                    printBoard(board)
                    print("Ai mutat: ", x, '\n')

        else:
            best = 9999
            sarituri = sarituriAI(board)
            bestMove = []
            if(len(sarituri)==0):
                mutari = succesoriAI(board)
                for mutare in mutari:
                    copie = creeazaBoardNormal(board, mutare)
                    val = algoritm(copie, 'a', 0, adancimeMax)
                    if(val<best):
                        bestMove = mutare
                        best = val
                board = creeazaBoardNormal(board, bestMove)
                printBoard(board)
                print("AI-ul a mutat: ", bestMove, '\n')
            else:
                while(len(sarituri)):
                    best = 9999
                    bestMove=[]
                    for saritura in sarituri:
                        copie = creeazaBoardSalt(board, saritura)
                        val = algoritm(copie, 'a', 0, adancimeMax)
                        if(val<best):
                            bestMove = saritura
                            best = val
                    board = creeazaBoardSalt(board, bestMove)
                    sarituri = sarituriAI(board)
                    for saritura in sarituri:
                        if (saritura[0], saritura[1]) != (x[2], x[3]):
                            sarituri.remove(saritura)
                    printBoard(board)
                    print("AI-ul a mutat: ", bestMove, f'si a luat piesa de pe pozitia ({bestMove[4]},{bestMove[5]})\n')
        player = 'n' if player == 'a' else 'a'
    if(final(board)=='a'):
        print("Ati castigat !")
        input()
        start()
    if(final(board)=='n'):
        print("Ati pierdut ...")
        input()
        start()

# **7. Pornirea jocului**

Inainte de inceperea jocului am prezentat regulie si am ales tipul algoritmului si adancimea maxima admisa.

In [ ]:
def start():
    algoritm = minimaxAlphaBetaLimitat
    adancimeMax = 5
    print(" \n\nREGULILE JOCULUI:")
    print("      -Orice salt posibil trebuie facut obligatoriu")
    print("      -Este posibil multi saltul si acesta este de asemenea, obligatoriu")
    print("      -Daca o piesa 'a' ajunge pe cealalta parte a tablei, aceasta devine 'A'(considerata rege) si se poate muta si inapoi")
    print("      -Exemplu de mutare: 1 2 2 3 a. Explicatie: piesa de pe pozitia (1,2) se muta pe pozitia (2,3) si este de tipul 'a'")
    print("      -In cazul regilor se precizeaza si piesa pe care o ia, exemplu: 1 2 3 4 2 3 A (piesa de pe pozitia (1,2) se duce pe pozitia(3,4) si ia piesa(2,3))")
    print("      -Lista de mutari disponibile va fi afisata la fiecare pas")
    x=input("   \nAPASATI ORICE PENTRU A INCEPE JOCUL... ")
    joc('a', algoritm, adancimeMax)
start()

 

REGULILE JOCULUI:
      -Orice salt posibil trebuie facut obligatoriu
      -Este posibil multi saltul si acesta este de asemenea, obligatoriu
      -Daca o piesa 'a' ajunge pe cealalta parte a tablei, aceasta devine 'A'(considerata rege) si se poate muta si inapoi
      -Exemplu de mutare: 1 2 2 3 a. Explicatie: piesa de pe pozitia (1,2) se muta pe pozitia (2,3) si este de tipul 'a'
      -In cazul regilor se precizeaza si piesa pe care o ia, exemplu: 1 2 3 4 2 3 A (piesa de pe pozitia (1,2) se duce pe pozitia(3,4) si ia piesa(2,3))
      -Lista de mutari disponibile va fi afisata la fiecare pas
    0    1    2    3    4    5    6    7
  ----------------------------------------
0|  -    n    -    n    -    n    -    n  |
1|  n    -    n    -    n    -    n    -  |
2|  -    n    -    n    -    n    -    n  |
3|  -    -    -    -    -    -    -    -  |
4|  -    -    -    -    -    -    -    -  |
5|  a    -    a    -    a    -    a    -  |
6|  -    a    -    a    -    a    -    a  |
7